In [1]:
import dataclasses
import json
from dataclasses import dataclass
from pathlib import Path

import ragas
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_correctness, faithfulness

from agents.basic_agents import BasicAgent
from agents.rag_agents import RagAgent

c:\Users\zhuwe\OneDrive\Desktop\projects\ragas_cookbook\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


this notebook contains code to try to implement an agent that evaluates the rag pipeline

starting with basic rag

In [2]:
from synthetic_data_generation.generate_syn_data import csv_to_list
from config.project_paths import synthetic_data_dir
import random
qna_dir = list(synthetic_data_dir.glob('*.csv'))
qnas = csv_to_list(qna_dir[0])
qnas = random.sample(qnas, 3)

In [5]:
rag_agent = RagAgent()

In [6]:
results = []

for question, answer in qnas:
    response = rag_agent.answer_with_rag(question)
    results.append((response, answer))

In [15]:
from utilities import print_long_text


print_long_text(results[2][0].answer.content)

Accessibility signal sounds and announcements are played when the primary cursor changes its line or the first time a
marker is added to the current line. They can be enabled automatically when a screen reader is attached and can be
controlled by the settings `accessibility.signals.*`. The commands **Help: List Signal Sounds** and **Help: List Signal
Announcements** inform the user of available sounds and announcements and allow for configuring their enabled/disabled
status.


the above code should be reformulated to able to form the dataset below and run ragas.evaluate

In [ ]:

data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts': [['The First AFL-NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'],
                 ['The Green Bay Packers...Green Bay, Wisconsin.', 'The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}

dataset = Dataset.from_dict(data_samples)


agent = BasicAgent()
score = evaluate(dataset,
                 metrics=[faithfulness, answer_correctness],
                 llm=agent.llm,
                 embeddings=agent.retriever.embedding_function,
                 )
score.to_pandas()

print(score)